<a href="https://colab.research.google.com/github/duddnd36/AI_assg/blob/main/AITEST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
myfile =files.upload()

Saving waterQuality1.csv to waterQuality1.csv


In [148]:
import io
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [168]:
#SVM 데이터셋
#데이터 셋 준비하기
data = pd.read_csv(io.BytesIO(myfile['waterQuality1.csv']))
data = data.drop('ammonia', axis = 1)
data = data.drop('flouride', axis = 1)
data = data.drop('selenium', axis = 1)
data = data.drop('uranium', axis = 1)

data = data[data['is_safe'] != '#NUM!']
data['is_safe'] = data['is_safe'].astype(int)

data.head(6)

,aluminium,arsenic,barium,cadmium,chloramine,chromium,copper,bacteria,viruses,lead,nitrates,nitrites,mercury,perchlorate,radium,silver,is_safe
0,1.65,0.04,2.85,0.007,0.35,0.83,0.17,0.20,0.000,0.054,16.08,1.13,0.007,37.75,6.78,0.34,1
1,2.32,0.01,3.31,0.002,5.28,0.68,0.66,0.65,0.650,0.100,2.01,1.93,0.003,32.26,3.21,0.27,1
2,1.01,0.04,0.58,0.008,4.24,0.53,0.02,0.05,0.003,0.078,14.16,1.11,0.006,50.28,7.07,0.44,0
3,1.36,0.04,2.96,0.001,7.23,0.03,1.66,0.71,0.710,0.016,1.41,1.29,0.004,9.12,1.72,0.45,1
4,0.92,0.03,0.20,0.006,2.67,0.69,0.57,0.13,0.001,0.117,6.74,1.11,0.003,16.90,2.41,0.06,1
5,0.94,0.03,2.88,0.003,0.80,0.43,1.38,0.67,0.670,0.135,9.75,1.89,0.006,27.17,5.42,0.19,1


In [ ]:
#데이터 셋 준비하기
data = pd.read_csv(io.BytesIO(myfile['waterQuality1.csv']))
data = data.rename(columns = {'aluminium':'알루미늄','ammonia':'암모니아','arsenic':'비소',
                              'barium':'바륨','cadmium':'카드뮴','chloramine':'클로라민',
                              'chromium':'크로뮴','copper':'구리','flouride':'불소',
                              'bacteria':'박테리아','viruses':'바이러스','lead':'납',
                              'nitrates':'질산염','nitrites':'아질산염','mercury':'수은',
                              'perchlorate':'과연소산염','radium':'라듐','selenium':'셀레늄',
                              'silver':'은','uranium':'우라늄','is_safe':'안전유무'})

In [ ]:

#아래의 인덱스를 가진 행에서 #NUM!이라는 string문자열이 들어가있음
data = data.drop([7551,7568,7890])
#string값이 있어 컬럼이 string타입으로 되어있어 float타입으로 변환
data = data.astype(float)

#안전 데이터 1개 추출
test_safe = data.iloc[321,:-1]
test_safe.astype(float)
test_safe = np.array([test_safe])

#비안전 데이터 1개 추출
test_unsafe = data.iloc[4312, :-1]
test_unsafe.astype(float)
test_unsafe = np.array([test_unsafe])

#안전, 비안전 데이터를 추출한 행 삭제
data = data.drop([321, 4312])

#학습데이터
x_data = data.iloc[:,:-1].astype(float)
y_data = data.iloc[:,20].astype(float)
pd.set_option('display.max_columns', 30)
data

,알루미늄,암모니아,비소,바륨,카드뮴,클로라민,크로뮴,구리,불소,박테리아,바이러스,납,질산염,아질산염,수은,과연소산염,라듐,셀레늄,은,우라늄,안전유무
0,1.65,9.08,0.04,2.85,0.007,0.35,0.83,0.17,0.05,0.20,0.000,0.054,16.08,1.13,0.007,37.75,6.78,0.08,0.34,0.02,1.0
1,2.32,21.16,0.01,3.31,0.002,5.28,0.68,0.66,0.90,0.65,0.650,0.100,2.01,1.93,0.003,32.26,3.21,0.08,0.27,0.05,1.0
2,1.01,14.02,0.04,0.58,0.008,4.24,0.53,0.02,0.99,0.05,0.003,0.078,14.16,1.11,0.006,50.28,7.07,0.07,0.44,0.01,0.0
3,1.36,11.33,0.04,2.96,0.001,7.23,0.03,1.66,1.08,0.71,0.710,0.016,1.41,1.29,0.004,9.12,1.72,0.02,0.45,0.05,1.0
4,0.92,24.33,0.03,0.20,0.006,2.67,0.69,0.57,0.61,0.13,0.001,0.117,6.74,1.11,0.003,16.90,2.41,0.02,0.06,0.02,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7994,0.05,7.78,0.00,1.95,0.040,0.10,0.03,0.03,1.37,0.00,0.000,0.197,14.29,1.00,0.005,3.57,2.13,0.09,0.06,0.03,1.0
7995,0.05,24.22,0.02,0.59,0.010,0.45,0.02,0.02,1.48,0.00,0.000,0.031,10.27,1.00,0.001,1.48,1.11,0.09,0.10,0.08,1.0
7996,0.09,6.85,0.00,0.61,0.030,0.05,0.05,0.02,0.91,0.00,0.000,0.182,15.92,1.00,0.000,1.35,4.84,0.00,0.04,0.05,1.0
7997,0.01,10.00,0.01,2.00,0.000,2.00,0.00,0.09,0.00,0.00,0.000,0.000,0.00,0.00,0.000,0.00,0.00,0.00,0.00,0.00,1.0


# 새 섹션 mlp

In [ ]:
print(x_data)
print(y_data)

      알루미늄   암모니아    비소    바륨    카드뮴  클로라민   크로뮴    구리    불소  박테리아   바이러스  \
0     1.65   9.08  0.04  2.85  0.007  0.35  0.83  0.17  0.05  0.20  0.000   
1     2.32  21.16  0.01  3.31  0.002  5.28  0.68  0.66  0.90  0.65  0.650   
2     1.01  14.02  0.04  0.58  0.008  4.24  0.53  0.02  0.99  0.05  0.003   
3     1.36  11.33  0.04  2.96  0.001  7.23  0.03  1.66  1.08  0.71  0.710   
4     0.92  24.33  0.03  0.20  0.006  2.67  0.69  0.57  0.61  0.13  0.001   
...    ...    ...   ...   ...    ...   ...   ...   ...   ...   ...    ...   
7994  0.05   7.78  0.00  1.95  0.040  0.10  0.03  0.03  1.37  0.00  0.000   
7995  0.05  24.22  0.02  0.59  0.010  0.45  0.02  0.02  1.48  0.00  0.000   
7996  0.09   6.85  0.00  0.61  0.030  0.05  0.05  0.02  0.91  0.00  0.000   
7997  0.01  10.00  0.01  2.00  0.000  2.00  0.00  0.09  0.00  0.00  0.000   
7998  0.04   6.85  0.01  0.70  0.030  0.05  0.01  0.03  1.00  0.00  0.000   

          납    질산염  아질산염     수은  과연소산염    라듐   셀레늄     은   우라늄  
0     0.05

In [ ]:

#모델 구성하기
tf.model = tf.keras.Sequential()
tf.model.add(tf.keras.layers.Dense(units = 10, input_dim = x_data.shape[1], activation = 'relu'))
tf.model.add(tf.keras.layers.Dense(units = 10, activation = 'relu'))
tf.model.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))
#모델 학습과정 설정하기
tf.model.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(lr = 0.01), metrics = ['accuracy'])
tf.model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 10)                110       
                                                                 
 dense_6 (Dense)             (None, 1)                 11        
                                                                 
Total params: 331
Trainable params: 331
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
#모델 학습시키기
history = tf.model.fit(x_data, y_data, epochs = 10, batch_size=64)
print("Accuracy : ", history.history['accuracy'][-1])

Epoch 1/10
125/125 [==============================] - 1s 3ms/step - loss: 0.3245 - accuracy: 0.8844
Epoch 2/10
125/125 [==============================] - 0s 2ms/step - loss: 0.2682 - accuracy: 0.8885
Epoch 3/10
125/125 [==============================] - 0s 3ms/step - loss: 0.2472 - accuracy: 0.9017
Epoch 4/10
125/125 [==============================] - 0s 3ms/step - loss: 0.2315 - accuracy: 0.9107
Epoch 5/10
125/125 [==============================] - 0s 3ms/step - loss: 0.2199 - accuracy: 0.9132
Epoch 6/10
125/125 [==============================] - 0s 3ms/step - loss: 0.2081 - accuracy: 0.9219
Epoch 7/10
125/125 [==============================] - 0s 2ms/step - loss: 0.2057 - accuracy: 0.9199
Epoch 8/10
125/125 [==============================] - 0s 2ms/step - loss: 0.2045 - accuracy: 0.9226
Epoch 9/10
125/125 [==============================] - 0s 2ms/step - loss: 0.1881 - accuracy: 0.9329
Epoch 10/10
125/125 [==============================] - 0s 2ms/step - loss: 0.1846 - accuracy: 0.9309

In [ ]:
evaluate = tf.model.evaluate(x_data, y_data)
print("loss: {0}, accuracy: {1}".format(evaluate[0], evaluate[1]*100))

250/250 [==============================] - 1s 2ms/step - loss: 0.1562 - accuracy: 0.9458
loss: 0.15616151690483093, accuracy: 94.58343982696533


# 새 섹션

deep

> 들여쓴 블록

# > 들여쓴 블록

---







In [125]:
#모델 구성하기


tf.model = tf.keras.Sequential()
tf.model.add(tf.keras.layers.Dense(units = 10, input_dim = x_data.shape[1], activation = 'relu')) #입력
tf.model.add(tf.keras.layers.Dense(units = 10, activation = 'relu')) #은닉
tf.model.add(tf.keras.layers.Dense(units = 5, activation = 'relu')) #은닉
tf.model.add(tf.keras.layers.Dense(units = 5, activation = 'relu')) # 은닉
tf.model.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid')) # 출력층
#모델 학습과정 설정하기
tf.model.compile(loss = 'mse', optimizer = tf.keras.optimizers.Adam(lr = 0.01), metrics = ['accuracy'])
history = tf.model.fit(x_data, y_data, epochs = 10,batch_size=64)
print("Accuracy : ", history.history['accuracy'][-1])
tf.model.summary()

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


125/125 [==============================] - 1s 2ms/step - loss: 0.1332 - accuracy: 0.8672
Epoch 2/10
125/125 [==============================] - 0s 2ms/step - loss: 0.1010 - accuracy: 0.8860
Epoch 3/10
125/125 [==============================] - 0s 3ms/step - loss: 0.1010 - accuracy: 0.8860
Epoch 4/10
125/125 [==============================] - 0s 3ms/step - loss: 0.1010 - accuracy: 0.8860
Epoch 5/10
125/125 [==============================] - 0s 2ms/step - loss: 0.1011 - accuracy: 0.8860
Epoch 6/10
125/125 [==============================] - 0s 3ms/step - loss: 0.1010 - accuracy: 0.8860
Epoch 7/10
125/125 [==============================] - 0s 2ms/step - loss: 0.1010 - accuracy: 0.8860
Epoch 8/10
125/125 [==============================] - 0s 2ms/step - loss: 0.1011 - accuracy: 0.8860
Epoch 9/10
125/125 [==============================] - 0s 2ms/step - loss: 0.1010 - accuracy: 0.8860
Epoch 10/10
125/125 [==============================] - 0s 2ms/step - loss: 0.1010 - accuracy: 0.8860
Accuracy :

In [126]:
evaluate = tf.model.evaluate(x_data, y_data, verbose = 0)
print("loss: {0}, accuracy: {1}".format(evaluate[0], evaluate[1]*100))


loss: 0.10097383707761765, accuracy: 88.60395550727844


# 새 섹션 svm

In [177]:
columns = dict(zip(list(range(data.shape[1])), data.columns.values.tolist()))
is_safe = data['is_safe']

data = pd.DataFrame(StandardScaler().fit_transform(data)).rename(columns = columns)
data['is_safe'] = is_safe
data = data.dropna()

In [203]:
X = data.drop('is_safe', axis = 1)
Y = data['is_safe']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 10)

In [204]:
parameters = {'C':[1, 10], 'gamma':('scale', 'auto')}

clf = GridSearchCV(SVC(), parameters)
clf.fit(X_train, Y_train)
predict = clf.predict(X_test)
clf.best_params_

{'C': 1, 'gamma': 'scale'}

In [205]:
print(classification_report(Y_test, predict))

              precision    recall  f1-score   support

         0.0       0.94      0.99      0.96      2330
         1.0       0.87      0.49      0.62       308

    accuracy                           0.93      2638
   macro avg       0.90      0.74      0.79      2638
weighted avg       0.93      0.93      0.92      2638



In [181]:
print(confusion_matrix(Y_test, predict))

[[2309   29]
 [ 215   86]]
